In [42]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
from matplotlib.gridspec import GridSpec
from scipy.ndimage import gaussian_filter
import subjects
import time

sessions = subjects.Sd().ratKday1
# sessions = subjects.Of().ratNday4


## Generate _basics.npy files

In [ ]:
for sub, sess in enumerate(sessions):
    # openephys_settingspath = (
    #     "/data/Clustering/SleepDeprivation/RatS/Day3SD/openEphysSettings/settings.xml"
    # )
    # sess.recinfo.generate_xml(settingsPath=openephys_settingspath)
    # sess.recinfo.makerecinfo(nShanks=[6, 8], skulleeg=[51], motion=[192, 193, 194])
    sess.recinfo.makerecinfo(nShanks=8)
    # sess.recinfo.makerecinfo(nShanks=8)
    print(sess.recinfo)

## Generate probemap and .prb for spyking circus

In [ ]:
# region
for sub, sess in enumerate(sessions):

    # sess.trange = np.array([])
    sess.recinfo.probemap.create(xypitch=((16.5, 15), (16, 15)))
    # sess.recinfo.probemap.plot()
    sess.recinfo.probemap.for_spyking_circus(rmv_badchans=True, shanksCombine=True)

# endregion


## artifacts file gen

In [ ]:
# region
for sub, sess in enumerate(sessions):
    # lfp = sess.recinfo.geteeg(chans=64)
    # plt.plot(np.linspace(0, len(lfp) / (1250 * 60), len(lfp)), lfp)
    zsc = sess.artifact.usingZscore(chans=[28, 64], thresh=7)
    sess.artifact.export2neuroscope()
    sess.artifact.plot()
# endregion


## Generate position files

In [ ]:
# region
for sub, sess in enumerate(sessions):
    sess.position.getPosition(method="from_metadata", scale=1.0)
    # sess.position.export2Neuroscope()

# endregion


## create epochs

In [ ]:
# region
for sub, sess in enumerate(sessions):
    sess.trange = np.array([])
    epochs = {
        "pre": [0, 11066],
        "maze1": [11070, 13970],
        "maze2": [20756, 24004],
        # "sd": [6033, 14818 + 5 * 3600],
        "post1": [13972, 20754],
        "post2": [24006, 42305],
        # "maze2": [36902, 40233],
    }
    sess.epochs.make_epochs(new_epochs=epochs)
    # sess.epochs.getfromCSV()

# endregion


## Create tracks

In [ ]:
# region
for sub, sess in enumerate(sessions):
    # sess.trange = np.array([])
    # sess.tracks.create(epoch_names=["maze1", "maze2"])
    sess.tracks.linearize_position(track_names=["maze2"], sample_sec=5)
    sess.tracks.estimate_run_laps("maze1", speedthresh=(18, 20))
    # sess.epochs.getfromCSV()

# endregion



## Gen spikes or instantenous firing rate

In [ ]:
# region
for sub, sess in enumerate(sessions):
    # sess.trange = np.array([])
    # sess.recinfo.sampfreq = 30000
    sess.spikes.from_Phy(
        folder="/data/Clustering/SleepDeprivation/RatS/Day5TwoNovel/spykcirc/RatS-Day5TwoNovel-2020-12-04_07-55-09-1.GUI/",
        fileformat="same_folder",
    )
    # sess.spikes.gen_instfiring()
# endregion


## label cells

In [ ]:
# region
for sub, sess in enumerate(sessions):
    # sess.recinfo.sampfreq = 30000
    sess.spikes.label_celltype()


# endregion

## Generate bestThetachan

In [ ]:
# region

for sub, sess in enumerate(sessions):
    sess.trange = np.array([])
    sess.theta.detectBestChan()
# endregion


## Detect pbe

In [ ]:
# region
for sub, sess in enumerate(sessions):
    sess.pbe.detect()


# endregion


## Detect ripples

In [ ]:
for sub,sess in sessions:
    # sess.ripple.channels()
    # sess.ripple.detect()
    sess.ripple.events

## Detect sleep states

In [43]:
for sub,sess in enumerate(sessions):
    sess.brainstates.detect()
    # sess.epochs.maze


starting emg calculation
emg calculation done
channel for sleep detection: 39
spectral properties calculated
RatK_Day1_2019-08-06_03-44-01.brainstates.npy created


In [47]:
sess.brainstates.epochs

,start,stop,duration,label
0,0.5,2.1,1.6,nrem
1,2.1,115.7,113.6,quiet
2,115.7,540.5,424.8,active
3,540.5,823.7,283.2,quiet
4,823.7,882.1,58.4,active
...,...,...,...,...
367,49950.1,50599.7,649.6,active
368,50599.7,50762.1,162.4,quiet
369,50762.1,50816.5,54.4,active
370,50816.5,50970.9,154.4,quiet
